In [8]:
import os
import openai
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
import timeit
persist_directory = '/Users/linqianpeng/Documents/immigration/docs/chroma'

openai_api_key = os.environ.get('OPENAI_API_KEY')

print(len(openai_api_key))

51


In [4]:
embedding = OpenAIEmbeddings()
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

print(vectordb._collection.count())

144


In [12]:
qa_stuff=RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=vectordb.as_retriever())

qa_refine = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    retriever=vectordb.as_retriever(),return_source_documents=True,
    chain_type="refine"
)

In [9]:
prompt_template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

  Instruction: 
  You are a immigration lawyer and your job is to answer questions on common law permanent resident (PR) application in Canada. 
  Use only information in the following paragraphs to answer the question at the end. \
  Explain the answer with reference to these paragraphs. If you don't know, say that you do not know.

  {context}
 
  Question: {question}

  Response:
  """
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

In [10]:
llm_name= "gpt-3.5-turbo"

llm = ChatOpenAI(model_name = llm_name,temperature=0)

In [ ]:
question = "how long does it take to process my PR application?"

result = qa_refine({"query": question})


In [32]:
print(result['result'])



In the case of a spousal sponsorship, it typically takes 39 days for processing a PR application through the PNP + Express Entry route. However, processing times can vary depending on the individual situation. The standard processing for a spousal sponsorship is officially 80% of applications completed in 12 months, though many people have seen much quicker turnarounds recently. A benefit of the spousal sponsorship is that you can submit your application as soon as it's filled out, whereas with PNP + Express Entry you'll need to submit an EOI and then wait for an ITA (and eventually wait for an ITA after receiving the nomination). In this case, it may be best to call the relevant office to inquire about the status of your application, and to check if you need to resubmit any required documents. Despite the fact that your application is stuck at "in review" status, it is likely that your application will not be denied if it takes longer than 30 days to hear back from them. If you feel

In [36]:
result_noreturn=qa_stuff({"query": question})

In [37]:
print(result['result'])



In the case of a spousal sponsorship, it typically takes 39 days for processing a PR application through the PNP + Express Entry route. However, processing times can vary depending on the individual situation. The standard processing for a spousal sponsorship is officially 80% of applications completed in 12 months, though many people have seen much quicker turnarounds recently. A benefit of the spousal sponsorship is that you can submit your application as soon as it's filled out, whereas with PNP + Express Entry you'll need to submit an EOI and then wait for an ITA (and eventually wait for an ITA after receiving the nomination). In this case, it may be best to call the relevant office to inquire about the status of your application, and to check if you need to resubmit any required documents. Despite the fact that your application is stuck at "in review" status, it is likely that your application will not be denied if it takes longer than 30 days to hear back from them. If you feel

In [40]:
%%time

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="stuff"
)

result = qa_chain({"query": question})

CPU times: user 36.1 ms, sys: 7.26 ms, total: 43.3 ms
Wall time: 5.74 s


In [41]:
print(result['result'])

I don't know the specific processing time for your PR application. Processing times can vary depending on various factors such as the type of application, the current workload of the immigration office, and individual circumstances. It's best to check the official government website or contact the relevant immigration authority for the most accurate and up-to-date information on processing times.


In [43]:
%%time
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="refine"
)

result = qa_chain({"query": question})


CPU times: user 52.6 ms, sys: 9.25 ms, total: 61.8 ms
Wall time: 36.2 s


In [ ]:
print(result['result'])

In [76]:
%%time
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),return_source_documents=True,
    chain_type="stuff"
)

result = qa_chain({"query": question})


CPU times: user 34.9 ms, sys: 10.7 ms, total: 45.7 ms
Wall time: 10.6 s


In [87]:
for i in (result['source_documents']):
    print(i['content'])
    print()

TypeError: 'Document' object is not subscriptable

In [93]:
print(i.page_content)

If you're concerned, you can always select 'Other' and upload a letter of explanation.
question_length: 634
answer_length: 346


In [96]:
i.metadata

{'row': 53, 'source': 'ImmigrationCanada_Sponsorship_QA_may7.csv'}

Document(page_content="If you're concerned, you can always select 'Other' and upload a letter of explanation.\nquestion_length: 634\nanswer_length: 346", metadata={'row': 53, 'source': 'ImmigrationCanada_Sponsorship_QA_may7.csv'})

In [47]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [49]:
from langchain.chains import ConversationalRetrievalChain
retriever=vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory)

In [52]:
%%time

question = "how long does it take to process my PR application?"

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),return_source_documents=True,
    chain_type="stuff"
)

result = qa({"question": question})

CPU times: user 25.2 ms, sys: 5.22 ms, total: 30.4 ms
Wall time: 10.9 s


In [53]:
print(result)

{'question': 'how long does it take to process my PR application?', 'chat_history': [HumanMessage(content='how long does it take to process my PR application?', additional_kwargs={}, example=False), AIMessage(content="I don't know the specific processing time for your PR application. Processing times can vary depending on various factors such as the type of application, the current workload of the immigration office, and individual circumstances. It's best to check the official government website or contact the relevant immigration authority for the most accurate and up-to-date information on processing times.", additional_kwargs={}, example=False)], 'answer': "I don't know the specific processing time for your PR application. Processing times can vary depending on various factors such as the type of application, the current workload of the immigration office, and individual circumstances. It's best to check the official government website or contact the relevant immigration authority 

In [55]:
print(result['answer'])

I don't know the specific processing time for your PR application. Processing times can vary depending on various factors such as the type of application, the current workload of the immigration office, and individual circumstances. It's best to check the official government website or contact the relevant immigration authority for the most accurate and up-to-date information on processing times.


In [56]:
question = "can you give me an range based on other's experiences?"
result = qa({"question": question})

In [57]:
print(result['answer'])

The processing times for PR applications can vary greatly depending on various factors such as the type of application, the country of residence, and the current workload of the immigration authorities. Generally, the standard processing time for a spousal sponsorship application is officially 80% of applications completed in 12 months. However, many people have been experiencing quicker turnarounds recently. On the other hand, the processing time for a Provincial Nominee Program (PNP) application through Express Entry can be shorter, but it still varies from case to case. It's important to note that these are just general estimates and individual processing times may differ.


In [67]:
print(result['answer'])

The processing times for PR applications can vary greatly depending on various factors such as the type of application, the country of residence, and the current workload of the immigration authorities. Generally, the standard processing time for a spousal sponsorship application is officially 80% of applications completed in 12 months. However, many people have been experiencing quicker turnarounds recently. On the other hand, the processing time for a Provincial Nominee Program (PNP) application through Express Entry can be shorter, but it still varies from case to case. It's important to note that these are just general estimates and individual processing times may differ.


In [68]:
print(result['question'])

can you give me an range based on other's experiences?


In [59]:
list(result)

['question', 'chat_history', 'answer']

In [65]:
result['chat_history'][0]

HumanMessage(content='how long does it take to process my PR application?', additional_kwargs={}, example=False)

In [66]:
result['chat_history'][1]

AIMessage(content="I don't know the specific processing time for your PR application. Processing times can vary depending on various factors such as the type of application, the current workload of the immigration office, and individual circumstances. It's best to check the official government website or contact the relevant immigration authority for the most accurate and up-to-date information on processing times.", additional_kwargs={}, example=False)

In [72]:
for i in result['chat_history']:
    print(i)
    print()

content='how long does it take to process my PR application?' additional_kwargs={} example=False

content="I don't know the specific processing time for your PR application. Processing times can vary depending on various factors such as the type of application, the current workload of the immigration office, and individual circumstances. It's best to check the official government website or contact the relevant immigration authority for the most accurate and up-to-date information on processing times." additional_kwargs={} example=False

content="can you give me an range based on other's experiences?" additional_kwargs={} example=False

content="The processing times for PR applications can vary greatly depending on various factors such as the type of application, the country of residence, and the current workload of the immigration authorities. Generally, the standard processing time for a spousal sponsorship application is officially 80% of applications completed in 12 months. However

In [75]:
i.content

"The processing times for PR applications can vary greatly depending on various factors such as the type of application, the country of residence, and the current workload of the immigration authorities. Generally, the standard processing time for a spousal sponsorship application is officially 80% of applications completed in 12 months. However, many people have been experiencing quicker turnarounds recently. On the other hand, the processing time for a Provincial Nominee Program (PNP) application through Express Entry can be shorter, but it still varies from case to case. It's important to note that these are just general estimates and individual processing times may differ."